In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, cohen_kappa_score


import sys
import os


parent_dir = os.path.dirname(os.getcwd())  
sys.path.append(parent_dir)  

from src.preprocessing import essay_to_sentences, clean_sentence
from src.lstm_model import build_lstm_model, reshape_for_lstm


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vijay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Load training data
train_df = pd.read_csv(
    "C:/Users/Vijay/Apy/A_LSTM-BERT/data/training_set_rel3.tsv",
    sep="\t",
    encoding="ISO-8859-1"
)

train_df = train_df[["essay", "domain1_score"]].dropna()

# Load test data (NO labels)
test_df = pd.read_csv(
    "C:/Users/Vijay/Apy/A_LSTM-BERT/data/test_set.tsv",
    sep="\t",
    encoding="ISO-8859-1"
)

test_df = test_df[["essay"]].dropna()

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)


Train shape: (12976, 2)
Test shape: (4254, 1)


In [3]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df["essay"],
    train_df["domain1_score"],
    test_size=0.2,
    random_state=42
)

train_essays = X_train.tolist()
val_essays = X_val.tolist()
test_essays = test_df["essay"].tolist()


In [4]:
train_sentences = []
for essay in train_essays:
    train_sentences.extend(essay_to_sentences(essay))


In [5]:
WORD_DIM = 300

w2v_model = Word2Vec(
    sentences=train_sentences,
    vector_size=WORD_DIM,
    window=10,
    min_count=40,
    workers=4,
    sample=1e-3,
    seed=42
)

w2v_model.wv.save_word2vec_format(
    "C:/Users/Vijay/Apy/A_LSTM-BERT/models/word2vec_lstm.bin",
    binary=True
)


In [6]:
def essay_to_vector(words, model, dim):
    valid_words = [w for w in words if w in model.wv]
    if not valid_words:
        return np.zeros(dim)
    return np.mean(model.wv[valid_words], axis=0)


def vectorize_essays(essays, model, dim):
    vectors = np.zeros((len(essays), dim))
    for i, essay in enumerate(essays):
        tokens = clean_sentence(essay)
        vectors[i] = essay_to_vector(tokens, model, dim)
    return vectors


In [7]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

In [8]:
X_train_vec = vectorize_essays(train_essays, w2v_model, WORD_DIM)
X_val_vec   = vectorize_essays(val_essays, w2v_model, WORD_DIM)
X_test_vec  = vectorize_essays(test_essays, w2v_model, WORD_DIM)

X_train_vec = reshape_for_lstm(X_train_vec)
X_val_vec   = reshape_for_lstm(X_val_vec)
X_test_vec  = reshape_for_lstm(X_test_vec)


In [9]:
lstm_model = build_lstm_model(WORD_DIM)

lstm_model.fit(
    X_train_vec,
    y_train,
    validation_data=(X_val_vec, y_val),
    batch_size=64,
    epochs=100,
    verbose=1
)


Epoch 1/100
163/163 [==============================] - 12s 37ms/step - loss: 54.1376 - mae: 3.8145 - val_loss: 33.4802 - val_mae: 2.6101
Epoch 2/100
163/163 [==============================] - 5s 32ms/step - loss: 29.6280 - mae: 2.5769 - val_loss: 22.4153 - val_mae: 2.2247
Epoch 3/100
163/163 [==============================] - 5s 33ms/step - loss: 21.5384 - mae: 2.3009 - val_loss: 14.7706 - val_mae: 1.8879
Epoch 4/100
163/163 [==============================] - 5s 32ms/step - loss: 14.8463 - mae: 2.0372 - val_loss: 11.4443 - val_mae: 1.7450
Epoch 5/100
163/163 [==============================] - 5s 32ms/step - loss: 11.6883 - mae: 1.8711 - val_loss: 8.0119 - val_mae: 1.5097
Epoch 6/100
163/163 [==============================] - 5s 33ms/step - loss: 10.0910 - mae: 1.7442 - val_loss: 7.3775 - val_mae: 1.4814
Epoch 7/100
163/163 [==============================] - 6s 35ms/step - loss: 9.3025 - mae: 1.6770 - val_loss: 6.4414 - val_mae: 1.4041
Epoch 8/100
163/163 [==============================

163/163 [==============================] - 8s 48ms/step - loss: 5.0509 - mae: 1.2318 - val_loss: 4.8518 - val_mae: 1.1704
Epoch 62/100
163/163 [==============================] - 9s 55ms/step - loss: 5.0222 - mae: 1.2222 - val_loss: 5.1077 - val_mae: 1.1908
Epoch 63/100
163/163 [==============================] - 7s 44ms/step - loss: 5.2970 - mae: 1.2539 - val_loss: 4.6305 - val_mae: 1.1364
Epoch 64/100
163/163 [==============================] - 7s 44ms/step - loss: 5.0288 - mae: 1.2323 - val_loss: 4.7836 - val_mae: 1.1585
Epoch 65/100
163/163 [==============================] - 7s 46ms/step - loss: 5.0801 - mae: 1.2395 - val_loss: 5.0232 - val_mae: 1.1808
Epoch 66/100
163/163 [==============================] - 8s 46ms/step - loss: 5.0783 - mae: 1.2321 - val_loss: 5.4689 - val_mae: 1.2164
Epoch 67/100
163/163 [==============================] - 7s 43ms/step - loss: 4.9565 - mae: 1.2227 - val_loss: 4.8304 - val_mae: 1.1460
Epoch 68/100
163/163 [==============================] - 8s 49ms/step

In [11]:
y_val_pred = lstm_model.predict(X_val_vec)
y_val_round = np.round(y_val_pred).astype(int)

rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
kappa = cohen_kappa_score(y_val, y_val_round)

print("Validation RMSE:", rmse)
print("Validation QWK:", kappa)


82/82 [==============================] - 1s 7ms/step
Validation RMSE: 2.2019970376466884
Validation QWK: 0.34304723908269186


In [12]:
test_predictions = lstm_model.predict(X_test_vec)
test_predictions = np.round(test_predictions).astype(int)

test_df["predicted_score"] = test_predictions
test_df.head()


133/133 [==============================] - 1s 7ms/step


,essay,predicted_score
0,I believe that computers have a positive effec...,9
1,"Dear @CAPS1, I know some problems have came up...",10
2,"Dear to whom it @MONTH1 concern, Computers are...",7
3,"Dear @CAPS1 @CAPS2, @CAPS3 has come to my atte...",10
4,"Dear Local newspaper, I think that people have...",9


In [13]:
test_df.to_csv(
    "C:/Users/Vijay/Apy/A_LSTM-BERT/results/test_set_predictions_lstm.csv",
    index=False
)
